## Dicom to JPG

In [ ]:
!python -V

Python 3.7.11


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/siim

/content/drive/MyDrive/siim


In [ ]:
!ls

competition  dataset  siim-covid19-detection  tmp  version1


In [ ]:
## !mkdir version1

In [ ]:
%cd version1

/content/drive/MyDrive/siim/version1


In [ ]:
'''
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!pip install -r requirements.txt
'''

'\n!git clone https://github.com/ultralytics/yolov5  # clone repo\n%cd yolov5\n!pip install -r requirements.txt\n'

In [ ]:
# Access WANDB account
!pip install -q --upgrade wandb
 
import wandb
wandb.login()

wandb: Currently logged in as: jutraman (use `wandb login --relogin` to force relogin)


True

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import pylibjpeg

In [ ]:
## data preparing

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/siim/siim-covid19-detection/train_image_level.csv')
path = '/content/drive/MyDrive/siim/siim-covid19-detection/train/ae3e63d94c13/288554eb6182/e00f9fe0cce5.dcm'
dicom = pydicom.read_file(path)

In [ ]:
'''
image_id = []
dim0 = []
dim1 = []
splits = []

tuples = []
for split in ['train', 'test']:
    save_dir = f'/content/drive/MyDrive/siim/tmp/{split}/'

    os.makedirs(save_dir, exist_ok=True)
    
    for dirname, _, filenames in tqdm(os.walk(f'/content/drive/MyDrive/siim/siim-covid19-detection/{split}')):
        for file in filenames:
            tuples.append((file, dirname, save_dir, split))

import multiprocessing

def resize_and_save(t):
    file, dirname, save_dir, split = t
    # set keep_ratio=True to have original aspect ratio
    xray = read_xray(os.path.join(dirname, file))
    im = resize(xray, size=256) 
#     print(os.path.join(dirname, file), os.path.join(save_dir, file.replace('dcm', 'png')))
    im.save(os.path.join(save_dir, file.replace('dcm', 'jpg')))

    image_id.append(file.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)


pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
# for t in tqdm(tuples):
for _ in tqdm(pool.imap_unordered(resize_and_save, tuples), total=len(tuples)):
    pass
'''

"\nimage_id = []\ndim0 = []\ndim1 = []\nsplits = []\n\ntuples = []\nfor split in ['train', 'test']:\n    save_dir = f'/content/drive/MyDrive/siim/tmp/{split}/'\n\n    os.makedirs(save_dir, exist_ok=True)\n    \n    for dirname, _, filenames in tqdm(os.walk(f'/content/drive/MyDrive/siim/siim-covid19-detection/{split}')):\n        for file in filenames:\n            tuples.append((file, dirname, save_dir, split))\n\nimport multiprocessing\n\ndef resize_and_save(t):\n    file, dirname, save_dir, split = t\n    # set keep_ratio=True to have original aspect ratio\n    xray = read_xray(os.path.join(dirname, file))\n    im = resize(xray, size=256) \n#     print(os.path.join(dirname, file), os.path.join(save_dir, file.replace('dcm', 'png')))\n    im.save(os.path.join(save_dir, file.replace('dcm', 'jpg')))\n\n    image_id.append(file.replace('.dcm', ''))\n    dim0.append(xray.shape[0])\n    dim1.append(xray.shape[1])\n    splits.append(split)\n\n\npool = multiprocessing.Pool(processes=multiproc

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []

for split in ['train', 'test']:
    save_dir = f'/content/drive/MyDrive/siim/tmp/{split}/'

    os.makedirs(save_dir, exist_ok=True)
    
    for dirname, _, filenames in tqdm(os.walk(f'/content/drive/MyDrive/siim/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=256)  
            im.save(os.path.join(save_dir, file.replace('dcm', 'jpg')))

            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/numpy_handler.py:359: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)


0it [00:00, ?it/s]

In [ ]:
df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})
df.to_csv('/content/drive/MyDrive/siim/tmp/meta.csv', index=False)